In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import OneSidedSelection
from matplotlib import pyplot
from numpy import where
import pandas as pd
from imblearn.over_sampling import SVMSMOTE
from imblearn.pipeline import Pipeline
import pandas as pd
from collections import Counter
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import NearMiss
import os

In [ ]:

df = pd.read_csv("./Downloads/final_data.csv")
df.drop('Unnamed: 0',axis=1,inplace=True)
print('Landslides are', round(df['y'].value_counts()[1]/len(df) * 100,2), '% of the dataset')
print('No Landslides are', round(df['y'].value_counts()[0]/len(df) * 100,2), '% of the dataset')

In [ ]:
df.loc[((df['aspect']<=22.5) | (df['aspect']>=337.5)),'Aspect']='n'
df.loc[((df['aspect']>22.5) & (df['aspect']<=67.5)),'Aspect']='ne'
df.loc[((df['aspect']>67.5) & (df['aspect']<=112.5)),'Aspect']='e'
df.loc[((df['aspect']>112.5) & (df['aspect']<=157.5)),'Aspect']='se'
df.loc[((df['aspect']>157.5) & (df['aspect']<=202.5)),'Aspect']='s'
df.loc[((df['aspect']>247.5) & (df['aspect']<=292.5)),'Aspect']='w'
df.loc[((df['aspect']>292.5) & (df['aspect']<=337.5)),'Aspect']='nw'
df.loc[((df['aspect']>202.5) & (df['aspect']<=247.5)),'Aspect']='sw'

In [ ]:
from category_encoders.james_stein import JamesSteinEncoder
JSE_encoder = JamesSteinEncoder()
train_jse=pd.DataFrame()
train_jse['as']= JSE_encoder.fit_transform(df.Aspect, df.y)
df=pd.concat([df,train_jse], axis=1)
df.drop(['Aspect','aspect'],inplace=True,axis=1)

In [ ]:
df=df.sample(frac=1, random_state=42)
df_test = pd.concat([df.loc[df['y'] == 1][:2000],df.loc[df['y'] == 0][:200000]])
df_test=df_test.sample(frac=1, random_state=42)
df_test.to_csv('./test.csv')
df_val=pd.concat([df.loc[df['y'] == 1][2000:4000],df.loc[df['y'] == 0][200000:400000]])
df_val=df_val.sample(frac=1, random_state=42)
df_val.to_csv('./val.csv')
df_train= pd.concat([df.loc[df['y'] == 1][4000:],df.loc[df['y'] == 0][400000:]])

In [ ]:
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200,n_jobs=17)
# transform the dataset
X, y = undersample.fit_resample(df_train.drop('y',axis=1), df_train['y'])
X['y']=y
X.to_csv('./train_OSS.csv')
over = SVMSMOTE(sampling_strategy=0.2,n_jobs=14)
train=pd.read_csv('./train_OSS.csv')
pipeline = Pipeline(steps=[('o', over), ('u', under)])
X_train, y_train = pipeline.fit_resample(train.drop('y',axis=1), train['y'])
print('Resample dataset shape', Counter(y_train))

In [ ]:
X_train['y']=y_train
X_train.to_csv('./final1.csv')